In [1]:
print("test")

test


In [40]:
import re
sample_file_name = "../dutch-language/data/sample-sentences-nlwiki.txt"
short_file_name = "../dutch-language/data/short-sentences-nlwiki.txt"
full_file_name = "../dutch-language/data/clean-sentences-nlwiki.txt"
corrected_file_name = "../dutch-language/data/corrected-sentences-nlwiki.txt"

file_name = corrected_file_name

regex_worden = re.compile(r'(.+ z|Z)end\b.*')
with open(file_name) as file:
    total_count = 0
    match_count = 0

    for line in file:
        if (total_count >= 3000000):
            break
        total_count += 1
        clean_line = line.rstrip()
        if (regex_worden.match(clean_line)):
            match_count += 1
            print(clean_line)

    print(total_count)
    print(match_count)

Hij belooft hierbij de Heilige Geest te zenden en geeft ze de opdracht: Zoals de Vader Mij heeft uitgezonden, zo zend Ik jullie uit.
Enkel Turkse staats tv zend nog in Koerdisch uit.
De frequentiekeuze hangt echter ook af van benodigde zend- en ontvangstantenne, doordringbaarheid van de atmosfeer , al of niet gewenste doordringbaarheid van muren, enzovoort.
De laagfrequente zend- en ontvangstapparatuur waarmee de Itasca het vliegtuig had kunnen lokaliseren, had Earhart in Nieuw-Guinea achtergelaten.
Er kan gebruik worden gemaakt van twee antennes, één zend- en één ontvangstantenne, maar tegenwoordig gebruikt men veelal één antenne voor zowel het zenden als het ontvangen.
Ook werd de voorgeïoniseerde vacuümbuis die als zend/ontvangstschakelaar fungeerde, vervangen door een golfpijpsysteem, de Magic-T.
Op het moment dat de zender in werking is detecteert de zend/ontvangschakelaar dit en schakelt bijna tijdloos over naar de stand zenden.
Hierdoor zullen doelen die binnen de waarnemingshor

In [11]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = ClassificationModel(
    "bert", "outputs/", use_cuda=True
)
print(type(model))


<class 'simpletransformers.classification.classification_model.ClassificationModel'>


In [23]:
predictions, raw_outputs = model.predict([
    "Ik word nieuwsgierig.",
    "Hoe word je gevraagd?",
    "Wat wordt je gevraagd?",
    "Wat word je opdringerig, zeg!",

    "Ik zend een pakje.",
    "Ik zend je een pakje.",
    "Ik zend u een heel mooie trui.",
    "Je zendt het toch nog vandaag op, hoop ik.", #
    "Jij zendt een hele grote doos naar oma.",
    "Zend je dat straks uit?", #
    "Zend jij echt die fiets via de post?",
    "Welke baas zendt je nu helemaal naar London voor 1 klant?",

    # With spelling mistake
    "Ik wordt nieuwsgierig.",
    "Hoe wordt je gevraagd?",
    "Wat word je gevraagd?",
    "Wat wordt je opdringerig, zeg!",

    "Ik zendt een pakje.",
    "Ik zendt je een pakje.",
    "Ik zendt u een heel mooie trui.", #
    "Je zend het toch nog vandaag op, hoop ik.",
    "Jij zend een hele grote doos naar oma.",
    "Zendt je dat straks uit?", #
    "Zendt jij echt die fiets via de post?",
    "Welke baas zend je nu helemaal naar London voor 1 klant?",
])
print()
print(predictions)
print(raw_outputs)

# Results are (still) perfect for "worden"
# Results are invalid (random) for "zenden"
# => no transfer learning from "worden" to "zenden" ?
#
# [0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 0 1 0 0]
# [[ 5.028973   -4.6238155 ]
#  [ 5.015533   -4.553905  ]
#  [ 5.001874   -4.5051813 ]
#  [ 4.8724637  -4.4218674 ]

#  [ 1.6203022  -1.5378906 ]
#  [ 0.12602812 -0.5217313 ]
#  [ 0.61012554 -1.0169847 ]
#  [-2.5860054   2.0353255 ]
#  [ 0.16640593 -0.02767875]
#  [-1.3042836   0.9139959 ]
#  [ 3.6788774  -3.0821662 ]
#  [ 1.0925493  -0.8713497 ]

#  [-5.3947897   4.4733806 ]
#  [-5.344915    4.23907   ]
#  [-5.315748    4.102456  ]
#  [-5.1123905   3.8351293 ]

#  [ 0.61081254 -0.63338786]
#  [ 0.76848555 -0.9399009 ]
#  [-0.82137966  0.20406562]
#  [ 0.6587549  -0.63729113]
#  [ 0.17024972 -0.11657579]
#  [-4.6694126   3.7321732 ]
#  [ 2.6194382  -2.2857945 ]
#  [ 0.5402404  -0.7211295 ]]

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100%|██████████| 3/3 [00:00<00:00,  4.09it/s]
[0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 0 1 0 0]
[[ 5.028973   -4.6238155 ]
 [ 5.015533   -4.553905  ]
 [ 5.001874   -4.5051813 ]
 [ 4.8724637  -4.4218674 ]
 [ 1.6203022  -1.5378906 ]
 [ 0.12602812 -0.5217313 ]
 [ 0.61012554 -1.0169847 ]
 [-2.5860054   2.0353255 ]
 [ 0.16640593 -0.02767875]
 [-1.3042836   0.9139959 ]
 [ 3.6788774  -3.0821662 ]
 [ 1.0925493  -0.8713497 ]
 [-5.3947897   4.4733806 ]
 [-5.344915    4.23907   ]
 [-5.315748    4.102456  ]
 [-5.1123905   3.8351293 ]
 [ 0.61081254 -0.63338786]
 [ 0.76848555 -0.9399009 ]
 [-0.82137966  0.20406562]
 [ 0.6587549  -0.63729113]
 [ 0.17024972 -0.11657579]
 [-4.6694126   3.7321732 ]
 [ 2.6194382  -2.2857945 ]
 [ 0.5402404  -0.7211295 ]]

